In [599]:
import requests
import pandas as pd
import json
from dateutil import parser


In [600]:
API_KEY = "a2faf0838c2aacce44be67cea8b40c06-48d69bac26199a4d5850294881134c34"
ACCOUNT_ID = "101-004-21509763-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [601]:
session = requests.Session()

In [602]:
session.headers.update({
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {API_KEY}'
})

In [603]:
params = dict(
    count = 10,
    granularity = "H1"
)

In [604]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [605]:
response = session.get(url, params=None, data=None, headers=None)

In [606]:
response.status_code

200

In [607]:
data = response.json()

In [608]:
data.keys()

dict_keys(['instruments', 'lastTransactionID'])

In [609]:
instruments_list = data["instruments"]

In [610]:
len(instruments_list)

123

In [611]:
instruments_list[1].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [612]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [613]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i }

In [614]:
instruments_dict['AU200_AUD']

{'name': 'AU200_AUD',
 'type': 'CFD',
 'displayName': 'Australia 200',
 'pipLocation': 0,
 'displayPrecision': 1,
 'tradeUnitsPrecision': 1,
 'marginRate': '0.05'}

In [615]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [616]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data
    

In [617]:

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        prices = ['mid', 'bid', 'ask']
        ohlc = ['o', 'h', 'l', 'c']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
            
        final_data.append(new_dict)
    df = pd.DataFrame(final_data)
    return df


In [621]:
def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name)
    if len(data) == 0:
        print("No candles there", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"Pair name:{pair_name}, Granularity: {granularity}, Candle shape: {candles_df.shape[0]}, time: {candles_df.time.min()}")

In [622]:
code, data = fetch_candles("EUR_SEK", count=10, granularity="H1")
candles_df = get_candles_df(data)

In [623]:
create_data_file("EUR_SEK", count=100, granularity="H4")

Pair name:EUR_SEK, Granularity: H4, Candle shape: 100, time: 2023-10-12 05:00:00+00:00
